In [1]:
import shopify
import datetime
from datetime import date, timedelta, time, datetime
import pandas as pd
from process_data.process_adress import process_adress
import os

SINCE_NUM_DAYS = 2
UNTIL_NUM_DAYS = 0

LAPOSTE_COLUMNS = ['Raison sociale', 'Civilité', 'Nom', 'Prénom', 'Identité / Service / Etage',
 'Bâtiment / Immeuble', 'N° et libellé de voie', 'Lieu-dit / BP', 'Code Postal', 'Ville',
 'Mobile', 'Téléphone', 'Email', 'Référence Destinataire']


API_KEY_ORDER_EXPORT = os.environ.get('API_KEY_ORDER_EXPORT')
API_PASSWORD_ORDER_EXPORT = os.environ.get('API_PASSWORD_ORDER_EXPORT')
SHOP_NAME = 'iriscosmetics'
shop_url = "https://%s.myshopify.com/admin" % (SHOP_NAME)

shopify.ShopifyResource.set_user(API_KEY_ORDER_EXPORT)
shopify.ShopifyResource.set_password(API_PASSWORD_ORDER_EXPORT)
shopify.ShopifyResource.set_site(shop_url)
#The following code prints the amount of orders up in your shopify store in command prompt window
#shop = shopify.Shop.current()
def date_from_today(n):
    date_today = date.today()
    return str(datetime.combine(date_today, time.min) - timedelta(days = n, hours = 2))

def laposte_df(orders):
    laposte_df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
    for order in orders:
        dict_order = order.to_dict()
        shipping_data = dict_order['shipping_address']
        df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
        df['created_at'] = [dict_order['created_at']]
        df['Number'] = [str(dict_order['number'])]
        df['Nom'] = [shipping_data['last_name'].replace('  ', ' ').strip()]
        df['Prénom'] = shipping_data['first_name'].replace('  ', ' ').strip()
        df['N° et libellé de voie'] = shipping_data['address1'].replace('  ', ' ').strip()
        if 'address2' in shipping_data.keys():
            df['N° et libellé de voie'] = df['N° et libellé de voie'] + ' ' + shipping_data['address2']
        df['Code Postal'] = shipping_data['zip']
        df['Ville'] = shipping_data['city'].strip()
        df['Mobile'] = shipping_data['phone']
        df['Email'] = dict_order['email']
        laposte_df = pd.concat([laposte_df, df])
        laposte_df['Number'] = laposte_df['Number'].apply(lambda x: str(1000 + int(x)))
        laposte_df['Mobile'] = laposte_df['Mobile'].apply(lambda x: x.replace(' ', '') if x else x)
        laposte_df = laposte_df.reset_index(drop = True)
        laposte_df.index = [i+1 for i in range(len(laposte_df))]
        laposte_df = process_adress(laposte_df)
    return laposte_df

subdir = 'prepafacile_csv/from_api'
yesterday = str(date.today() - timedelta(1))

orders = shopify.Order.find(created_at_min = date_from_today(SINCE_NUM_DAYS),
                           created_at_max = date_from_today(UNTIL_NUM_DAYS))
print(len(orders))

if not os.path.exists(subdir):
    os.mkdir(subdir)
df_orders = laposte_df(orders)
print(df_orders[['Number', 'created_at']])
df_orders = df_orders.drop(['created_at', 'Number'], axis = 1)
df_orders.to_csv(subdir + '/' + yesterday +'.csv', sep = ';')


30


C:\Users\edmon\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



   Number                 created_at
1   30116  2019-06-10T23:23:19+02:00
2   29115  2019-06-10T22:46:38+02:00
3   28114  2019-06-10T20:55:37+02:00
4   27113  2019-06-10T19:59:19+02:00
5   26112  2019-06-10T18:42:49+02:00
6   25111  2019-06-10T16:47:15+02:00
7   24110  2019-06-10T16:33:19+02:00
8   23109  2019-06-10T13:16:42+02:00
9   22108  2019-06-10T12:28:34+02:00
10  21107  2019-06-10T10:30:05+02:00
11  20106  2019-06-10T08:46:53+02:00
12  19105  2019-06-10T08:13:59+02:00
13  18104  2019-06-10T08:04:53+02:00
14  17103  2019-06-10T02:22:27+02:00
15  16102  2019-06-10T01:39:20+02:00
16  15101  2019-06-09T23:59:27+02:00
17  14100  2019-06-09T23:47:04+02:00
18  13099  2019-06-09T23:45:03+02:00
19  12098  2019-06-09T22:01:25+02:00
20  11097  2019-06-09T19:33:08+02:00
21  10096  2019-06-09T17:34:38+02:00
22   9095  2019-06-09T16:50:11+02:00
23   8094  2019-06-09T16:32:16+02:00
24   7093  2019-06-09T13:37:27+02:00
25   6092  2019-06-09T10:28:58+02:00
26   5091  2019-06-09T09:32:56+02:00
2

In [18]:
df_orders

,Bâtiment / Immeuble,Civilité,Code Postal,Email,Identité / Service / Etage,Lieu-dit / BP,Mobile,Nom,N° et libellé de voie,Prénom,Raison sociale,Référence Destinataire,Téléphone,Ville
1,,NaN,92320,ellapinmarieline@hotmail.fr,NaN,,650110875,marie-line,38 avenue marcelin berthelot,ellapin,NaN,NaN,NaN,Châtillon 92
2,,NaN,33133,martine.asquini@gmail.com,NaN,,,asquini,7 Chemin de Pimpine,martine,NaN,NaN,NaN,Galgon
3,,NaN,06570,,NaN,,+33623853270,COTTO,2400 Route des Serres Villa l’Olivai,Monique,NaN,NaN,NaN,Saint-Paul-de-Vence
4,,NaN,24400,vioviomike@hotmail.fr,NaN,,656793041,barbieri,4 TER DE LA BOETIE,Violette,NaN,NaN,NaN,MUSSIDAN
5,,NaN,74380,angele.neuraz@gmail.com,NaN,,,Angele,114 chemin des moranches,Neuraz,NaN,NaN,NaN,Cranves SALES
6,,NaN,06100,josette.caldar@orange.fr,NaN,,0683334658,Caldari,101 Avenue Cyrnos,Josette,NaN,NaN,NaN,Nice
7,,NaN,14340,kolthoff@orange.fr,NaN,,None,Catharina,Cour de la Bruy�re25 C�te LaurentSt,Lundholm,NaN,NaN,NaN,CAMBREMER
8,,NaN,25650,florence1923@live.fr,NaN,,,Giroux,"12,bief de la charmotte",Florence,NaN,NaN,NaN,Montbenoit
9,,NaN,75013,marie.christinedavid5@gmail.com,NaN,,145816958,daavid,8 rue de pouy,marie,NaN,NaN,NaN,Paris
10,,NaN,14150,,NaN,,0670729923,DOS SANTOS,20 Avenue de la Résistance Chez mons,catherine,NaN,NaN,NaN,Ouistreham


In [27]:
orders[0].to_dict()['created_at']

'2019-06-09T00:04:18+02:00'

In [29]:
date.today()

datetime.date(2019, 6, 9)

In [37]:
datetime.strptime('2019-06-09T00:04:18+02:00')

SyntaxError: EOL while scanning string literal (<ipython-input-37-090332da7a5e>, line 1)

In [50]:
df_orders['Nom'][6] = 'Maelys Amedee'

In [66]:
df_orders.to_csv(subdir + '/' + yesterday +'.csv', sep = ';')

In [64]:
import random
df_orders['Poids'] = str(random.randint(20,100))
df_orders['Poids'] = df_orders['Poids'].apply(lambda x: str(random.randint(20,100)))

In [65]:
df_orders['Poids']

1      26
2      41
3      36
4     100
5      25
6      26
7      91
8      72
9      43
10     63
11     89
12     27
13     74
14     35
15     81
16     87
17     95
18     77
19     83
Name: Poids, dtype: object

In [67]:
'MSTest, JUnit, TestNG, NUni, DUnit,
PyUnit, Ruby, PHPUnit, TestComplete.split(' ')

['',
 'MSTest',
 'JUnit',
 'TestNG',
 'NUni',
 'DUnit',
 'PyUnit',
 'Ruby',
 'PHPUnit',
 'TestComplete']